In [3]:
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory

## Problem 1 Version 1

In [8]:
m = ConcreteModel()

m.GRADES = Set(initialize = ['1A', '1B', '2A'])
m.TIMESLOTS = Set(initialize = range(8, 16))

m.Assign = Var(m.GRADES, m.TIMESLOTS, domain = Binary)

def one_timeslot_per_grade(model, t):
    return quicksum(model.Assign[g, t] for g in model.GRADES) <= 1

m.One_Timeslot_per_grade = Constraint(m.TIMESLOTS, rule = one_timeslot_per_grade)
m.assign_sum = Objective(rule = lambda model: sum_product(model.Assign), sense = maximize)

In [9]:
opt = SolverFactory('glpk')
opt.solve(m, tee = True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmppdoc_8hg.glpk.raw --wglp /tmp/tmpbk6p5ey7.glpk.glp --cpxlp
 /tmp/tmp9wpjtaho.pyomo.lp
Reading problem data from '/tmp/tmp9wpjtaho.pyomo.lp'...
/tmp/tmp9wpjtaho.pyomo.lp:106: warning: lower bound of variable 'x2' redefined
/tmp/tmp9wpjtaho.pyomo.lp:106: warning: upper bound of variable 'x2' redefined
8 rows, 24 columns, 24 non-zeros
24 integer variables, all of which are binary
130 lines were read
Writing problem data to '/tmp/tmpbk6p5ey7.glpk.glp'...
91 lines were written
GLPK Integer Optimizer 5.0
8 rows, 24 columns, 24 non-zeros
24 integer variables, all of which are binary
Preprocessing...
8 rows, 24 columns, 24 non-zeros
24 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 8
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
8 

{'Problem': [{'Name': 'unknown', 'Lower bound': 8.0, 'Upper bound': 8.0, 'Number of objectives': 1, 'Number of constraints': 8, 'Number of variables': 24, 'Number of nonzeros': 24, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.005380392074584961}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [10]:
m.pprint()

2 Set Declarations
    GRADES : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'1A', '1B', '2A'}
    TIMESLOTS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    8 : {8, 9, 10, 11, 12, 13, 14, 15}

1 Var Declarations
    Assign : Size=24, Index=GRADES*TIMESLOTS
        Key        : Lower : Value : Upper : Fixed : Stale : Domain
         ('1A', 8) :     0 :   1.0 :     1 : False : False : Binary
         ('1A', 9) :     0 :   1.0 :     1 : False : False : Binary
        ('1A', 10) :     0 :   1.0 :     1 : False : False : Binary
        ('1A', 11) :     0 :   1.0 :     1 : False : False : Binary
        ('1A', 12) :     0 :   1.0 :     1 : False : False : Binary
        ('1A', 13) :     0 :   1.0 :     1 : False : False : Binary
        ('1A', 14) :     0 :   1.0 :     1 : False : False : Binary
        ('1A', 15) :     0 :   1.0 :     1 : 

In [18]:
d = {}
for g in m.GRADES:
    for t in m.TIMESLOTS:
        if value(m.Assign[g, t]):
            d[t] = g

In [25]:
pd.DataFrame(d.values(), index = d.keys())

,0
8,1A
9,1A
10,1A
11,1A
12,1A
13,1A
14,1A
15,1A


### Version 2

In [27]:
m = ConcreteModel()

m.GRADES = Set(initialize = ['1A', '1B', '2A'])
m.TIMESLOTS = Set(initialize = range(8, 16))

m.Assign = Var(m.GRADES, m.TIMESLOTS, domain = Binary)

def one_timeslot_per_grade(model, t):
    return quicksum(model.Assign[g, t] for g in model.GRADES) <= 1

m.One_Timeslot_per_grade = Constraint(m.TIMESLOTS, rule = one_timeslot_per_grade)
m.assign_sum = Objective(rule = lambda model: sum_product(model.Assign), sense = maximize)

def equality(model, g, g_prime):
    if g == g_prime:
        return Constraint.Skip
    return quicksum(model.Assign[g, t] for t in model.TIMESLOTS) == quicksum(model.Assign[g_prime, t] for t in model.TIMESLOTS)

m.equality_rule = Constraint(m.GRADES, m.GRADES, rule = equality)

In [31]:
opt.solve(m)
m.display()

Model unknown

  Variables:
    Assign : Size=24, Index=GRADES*TIMESLOTS
        Key        : Lower : Value : Upper : Fixed : Stale : Domain
         ('1A', 8) :     0 :   1.0 :     1 : False : False : Binary
         ('1A', 9) :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 10) :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 11) :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 12) :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 13) :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 14) :     0 :   1.0 :     1 : False : False : Binary
        ('1A', 15) :     0 :   0.0 :     1 : False : False : Binary
         ('1B', 8) :     0 :   0.0 :     1 : False : False : Binary
         ('1B', 9) :     0 :   1.0 :     1 : False : False : Binary
        ('1B', 10) :     0 :   0.0 :     1 : False : False : Binary
        ('1B', 11) :     0 :   0.0 :     1 : False : False : Binary
        ('1B', 12) :     0 :   0.0 :     1 

In [33]:
d = {}
for g in m.GRADES:
    for t in m.TIMESLOTS:
        if value(m.Assign[g, t]):
            d[t] = g
pd.DataFrame(d.values(), index = d.keys())


,0
8,1A
14,1A
9,1B
13,1B
10,2A
11,2A


#### Using the other constraint

In [34]:
m = ConcreteModel()

m.GRADES = Set(initialize = ['1A', '1B', '2A'])
m.TIMESLOTS = Set(initialize = range(8, 16))

m.Assign = Var(m.GRADES, m.TIMESLOTS, domain = Binary)

def one_timeslot_per_grade(model, t):
    return quicksum(model.Assign[g, t] for g in model.GRADES) <= 1

m.One_Timeslot_per_grade = Constraint(m.TIMESLOTS, rule = one_timeslot_per_grade)
m.assign_sum = Objective(rule = lambda model: sum_product(model.Assign), sense = maximize)

def equality(model, g, g_prime):
    if g == g_prime:
        return Constraint.Skip
    return quicksum(model.Assign[g, t] for t in model.TIMESLOTS) <= len(model.TIMESLOTS) / len(model.GRADES)

m.equality_rule = Constraint(m.GRADES, m.GRADES, rule = equality)

In [35]:
opt.solve(m)
m.display()

Model unknown

  Variables:
    Assign : Size=24, Index=GRADES*TIMESLOTS
        Key        : Lower : Value : Upper : Fixed : Stale : Domain
         ('1A', 8) :     0 :   1.0 :     1 : False : False : Binary
         ('1A', 9) :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 10) :     0 :   1.0 :     1 : False : False : Binary
        ('1A', 11) :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 12) :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 13) :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 14) :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 15) :     0 :   0.0 :     1 : False : False : Binary
         ('1B', 8) :     0 :   0.0 :     1 : False : False : Binary
         ('1B', 9) :     0 :   1.0 :     1 : False : False : Binary
        ('1B', 10) :     0 :   0.0 :     1 : False : False : Binary
        ('1B', 11) :     0 :   1.0 :     1 : False : False : Binary
        ('1B', 12) :     0 :   0.0 :     1 

In [36]:
d = {}
for g in m.GRADES:
    for t in m.TIMESLOTS:
        if value(m.Assign[g, t]):
            d[t] = g
pd.DataFrame(d.values(), index = d.keys())


,0
8,1A
10,1A
9,1B
11,1B
13,2A
14,2A


### Verion 3

In [40]:
m = ConcreteModel()

m.GRADES = Set(initialize = ['1A', '1B', '2A'])
m.TIMESLOTS = Set(initialize = range(8, 16))
m.ROOMS = Set(initialize = ['101', '102'])

m.Assign = Var(m.GRADES, m.TIMESLOTS, m.ROOMS, domain = Binary)

def one_grade_in_one_timeslot_and_one_room(model, t, r):
    return quicksum(model.Assign[g, t, r] for g in model.GRADES) <= 1

m.One_Timeslot_per_grade = Constraint(m.TIMESLOTS, m.ROOMS, rule = one_grade_in_one_timeslot_and_one_room)
m.assign_sum = Objective(rule = lambda model: sum_product(model.Assign), sense = maximize)

def equality(model, g, g_prime):
    if g == g_prime:
        return Constraint.Skip
    return quicksum(model.Assign[g, t, r] for t in model.TIMESLOTS for r in model.ROOMS) == quicksum(model.Assign[g_prime, t, r] for t in model.TIMESLOTS for r in model.ROOMS)

m.equality_rule = Constraint(m.GRADES, m.GRADES, rule = equality)

In [42]:
opt.solve(m, tee = True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpoiq53zc7.glpk.raw --wglp /tmp/tmph5cnur_t.glpk.glp --cpxlp
 /tmp/tmpjbovl7gm.pyomo.lp
Reading problem data from '/tmp/tmpjbovl7gm.pyomo.lp'...
/tmp/tmpjbovl7gm.pyomo.lp:412: warning: lower bound of variable 'x2' redefined
/tmp/tmpjbovl7gm.pyomo.lp:412: warning: upper bound of variable 'x2' redefined
22 rows, 48 columns, 240 non-zeros
48 integer variables, all of which are binary
460 lines were read
Writing problem data to '/tmp/tmph5cnur_t.glpk.glp'...
377 lines were written
GLPK Integer Optimizer 5.0
22 rows, 48 columns, 240 non-zeros
48 integer variables, all of which are binary
Preprocessing...
22 rows, 48 columns, 240 non-zeros
48 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 18
Solving LP relaxation...
GLPK Simplex Optimize

{'Problem': [{'Name': 'unknown', 'Lower bound': 15.0, 'Upper bound': 15.0, 'Number of objectives': 1, 'Number of constraints': 22, 'Number of variables': 48, 'Number of nonzeros': 240, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '131461', 'Number of created subproblems': '131461'}}, 'Error rc': 0, 'Time': 24.362550735473633}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [43]:
m.display()

Model unknown

  Variables:
    Assign : Size=48, Index=GRADES*TIMESLOTS*ROOMS
        Key               : Lower : Value : Upper : Fixed : Stale : Domain
         ('1A', 8, '101') :     0 :   1.0 :     1 : False : False : Binary
         ('1A', 8, '102') :     0 :   0.0 :     1 : False : False : Binary
         ('1A', 9, '101') :     0 :   0.0 :     1 : False : False : Binary
         ('1A', 9, '102') :     0 :   1.0 :     1 : False : False : Binary
        ('1A', 10, '101') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 10, '102') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 11, '101') :     0 :   1.0 :     1 : False : False : Binary
        ('1A', 11, '102') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 12, '101') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 12, '102') :     0 :   1.0 :     1 : False : False : Binary
        ('1A', 13, '101') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 13, '1

In [76]:
from collections import defaultdict
d = defaultdict(lambda: defaultdict(str))
for g in m.GRADES:
    for t in m.TIMESLOTS:
        for r in m.ROOMS:
            if value(m.Assign[g, t, r]):
                d[r][t] = g

In [82]:
pd.DataFrame(d).fillna('N/A').sort_index()

,101,102
8,1A,1B
9,1B,1A
10,2A,2A
11,1A,1B
12,1B,1A
13,2A,1A
14,1B,2A
15,N/A,2A


#### Uing alternate math condition

In [88]:
m = ConcreteModel()

m.GRADES = Set(initialize = ['1A', '1B', '2A'])
m.TIMESLOTS = Set(initialize = range(8, 16))
m.ROOMS = Set(initialize = ['101', '102'])

m.Assign = Var(m.GRADES, m.TIMESLOTS, m.ROOMS, domain = Binary)

def one_grade_in_one_timeslot_and_one_room(model, t, r):
    return quicksum(model.Assign[g, t, r] for g in model.GRADES) <= 1

def one_room_with_one_grade(model, g, t):
    return quicksum(model.Assign[g, t, r] for r in model.ROOMS) <= 1
    

m.One_Timeslot_per_grade = Constraint(m.TIMESLOTS, m.ROOMS, rule = one_grade_in_one_timeslot_and_one_room)
m.One_Room_with_one_grade = Constraint(m.GRADES, m.TIMESLOTS, rule = one_room_with_one_grade)

m.assign_sum = Objective(rule = lambda model: sum_product(model.Assign), sense = maximize)

def equality(model, g):
    return quicksum(model.Assign[g, t, r] for t in model.TIMESLOTS for r in model.ROOMS) <= len(model.TIMESLOTS) * len(model.ROOMS) / len(model.GRADES)

m.equality_rule = Constraint(m.GRADES, rule = equality)

In [89]:
opt.solve(m, tee = True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmp827lh3rx.glpk.raw --wglp /tmp/tmpjye83myf.glpk.glp --cpxlp
 /tmp/tmp5kvxtiuc.pyomo.lp
Reading problem data from '/tmp/tmp5kvxtiuc.pyomo.lp'...
/tmp/tmp5kvxtiuc.pyomo.lp:379: warning: lower bound of variable 'x2' redefined
/tmp/tmp5kvxtiuc.pyomo.lp:379: warning: upper bound of variable 'x2' redefined
43 rows, 48 columns, 144 non-zeros
48 integer variables, all of which are binary
427 lines were read
Writing problem data to '/tmp/tmpjye83myf.glpk.glp'...
329 lines were written
GLPK Integer Optimizer 5.0
43 rows, 48 columns, 144 non-zeros
48 integer variables, all of which are binary
Preprocessing...
43 rows, 48 columns, 144 non-zeros
48 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 43
Solving LP relaxation...
GLPK Simplex Optimize

{'Problem': [{'Name': 'unknown', 'Lower bound': 15.0, 'Upper bound': 15.0, 'Number of objectives': 1, 'Number of constraints': 43, 'Number of variables': 48, 'Number of nonzeros': 144, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '25191', 'Number of created subproblems': '25191'}}, 'Error rc': 0, 'Time': 3.2675940990448}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [90]:
m.display()

Model unknown

  Variables:
    Assign : Size=48, Index=GRADES*TIMESLOTS*ROOMS
        Key               : Lower : Value : Upper : Fixed : Stale : Domain
         ('1A', 8, '101') :     0 :   1.0 :     1 : False : False : Binary
         ('1A', 8, '102') :     0 :   0.0 :     1 : False : False : Binary
         ('1A', 9, '101') :     0 :   1.0 :     1 : False : False : Binary
         ('1A', 9, '102') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 10, '101') :     0 :   1.0 :     1 : False : False : Binary
        ('1A', 10, '102') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 11, '101') :     0 :   1.0 :     1 : False : False : Binary
        ('1A', 11, '102') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 12, '101') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 12, '102') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 13, '101') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 13, '1

In [92]:
from collections import defaultdict
d = defaultdict(lambda: defaultdict(str))
for g in m.GRADES:
    for t in m.TIMESLOTS:
        for r in m.ROOMS:
            if value(m.Assign[g, t, r]):
                d[r][t] = g

In [94]:
pd.DataFrame(d).fillna('N/A').sort_index()

,101,102
8,1A,2A
9,1A,N/A
10,1A,1B
11,1A,1B
12,2A,1B
13,1B,2A
14,2A,1B
15,1A,2A


### Verion 4 - with the constraint of len(TIMESLOTS) * len(ROOMS) / len(GRADES)

In [105]:
m = AbstractModel()

m.GRADES = Set()
m.TIMESLOTS = Set()
m.ROOMS = Set()

m.pop = Param(m.GRADES)
m.cap = Param(m.ROOMS)

m.Assign = Var(m.GRADES, m.TIMESLOTS, m.ROOMS, domain = Binary)

def one_grade_in_one_timeslot_and_one_room(model, t, r):
    return quicksum(model.Assign[g, t, r] for g in model.GRADES) <= 1

def one_room_with_one_grade(model, g, t):
    return quicksum(model.Assign[g, t, r] for r in model.ROOMS) <= 1

def capacity_of_room(model, g, t, r):
    return model.Assign[g, t, r] * model.pop[g] <= model.cap[r]

m.One_Timeslot_per_grade = Constraint(m.TIMESLOTS, m.ROOMS, rule = one_grade_in_one_timeslot_and_one_room)
m.One_Room_with_one_grade = Constraint(m.GRADES, m.TIMESLOTS, rule = one_room_with_one_grade)
m.capacity_of_room = Constraint(m.GRADES, m.TIMESLOTS, m.ROOMS, rule = capacity_of_room)

m.assign_sum = Objective(rule = lambda model: sum_product(model.Assign), sense = maximize)

# This does not work because, len(TIMESLOTS) * len(ROOMS) / len(GRADES) = 5.33
# This means that grade 1A could have a total time of 5, and grade 1B could have a total time of 0.
# 5.33 is the MAXIMUM amount of face-to-face time ONE grade can have, if all timeslots across all rooms were divided equally,
# but it doesn't assert that two grades should have the same face-to-face time.
def wrong_equality(model, g):
    return quicksum(model.Assign[g, t, r] for t in model.TIMESLOTS for r in model.ROOMS) <= len(model.TIMESLOTS) * len(model.ROOMS) / len(model.GRADES)


m.equality_rule = Constraint(m.GRADES, rule = equality)

In [106]:
instanceData = { None: {
    'GRADES': {None: ['1A', '1B', '2A']},
    'TIMESLOTS': {None: range(8, 16)},
    'ROOMS': {None: ['101', '102']},
    'pop': {'1A': 31, '1B': 36, '2A': 39},
    'cap': {'101': 35, '102': 50}
}}

In [107]:
instance = m.create_instance(instanceData)

In [108]:
opt.solve(instance, tee = True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmp5eae9woy.glpk.raw --wglp /tmp/tmpqv1c0vzk.glpk.glp --cpxlp
 /tmp/tmpqkwmoib0.pyomo.lp
Reading problem data from '/tmp/tmpqkwmoib0.pyomo.lp'...
/tmp/tmpqkwmoib0.pyomo.lp:571: warning: lower bound of variable 'x2' redefined
/tmp/tmpqkwmoib0.pyomo.lp:571: warning: upper bound of variable 'x2' redefined
91 rows, 48 columns, 192 non-zeros
48 integer variables, all of which are binary
619 lines were read
Writing problem data to '/tmp/tmpqv1c0vzk.glpk.glp'...
473 lines were written
GLPK Integer Optimizer 5.0
91 rows, 48 columns, 192 non-zeros
48 integer variables, all of which are binary
Preprocessing...
19 rows, 32 columns, 72 non-zeros
32 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 19
Solving LP relaxation...
GLPK Simplex Optimizer

{'Problem': [{'Name': 'unknown', 'Lower bound': 13.0, 'Upper bound': 13.0, 'Number of objectives': 1, 'Number of constraints': 91, 'Number of variables': 48, 'Number of nonzeros': 192, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.005919694900512695}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [109]:
instance.display()

Model unknown

  Variables:
    Assign : Size=48, Index=GRADES*TIMESLOTS*ROOMS
        Key               : Lower : Value : Upper : Fixed : Stale : Domain
         ('1A', 8, '101') :     0 :   1.0 :     1 : False : False : Binary
         ('1A', 8, '102') :     0 :   0.0 :     1 : False : False : Binary
         ('1A', 9, '101') :     0 :   1.0 :     1 : False : False : Binary
         ('1A', 9, '102') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 10, '101') :     0 :   1.0 :     1 : False : False : Binary
        ('1A', 10, '102') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 11, '101') :     0 :   1.0 :     1 : False : False : Binary
        ('1A', 11, '102') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 12, '101') :     0 :   1.0 :     1 : False : False : Binary
        ('1A', 12, '102') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 13, '101') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 13, '1

In [110]:
from collections import defaultdict
d = defaultdict(lambda: defaultdict(str))
for g in instance.GRADES:
    for t in instance.TIMESLOTS:
        for r in instance.ROOMS:
            if value(instance.Assign[g, t, r]):
                d[r][t] = g

In [111]:
pd.DataFrame(d).fillna('N/A').sort_index()

,101,102
8,1A,1B
9,1A,1B
10,1A,1B
11,1A,1B
12,1A,1B
13,N/A,2A
14,N/A,2A
15,N/A,2A


- In the above example, grade 1A and 1B got 5 timeslots, but grade 2A got 3
- Both of them satisfied the constraint $ \sum_{r} \sum_{t} {assign_{g,t,r}} <= 5.33 $
- $ | TIMESLOTS | * | ROOMS | $ denotes the total available face-to-face time
- $ | TIMESLOTS | * | ROOMS | / | GRADES| $ denotes the MAXIMUM total available face-to-face time for each grade, but it doesn't necessarily imply that the values of assign will reach that ceiling. The solution of (5, 0, 0) is also feasible, as per the above constraint.
- For the same reasons, $ | TIMESLOTS | / | GRADES | $ in version 2 of the problem is a wrong constraint. It works in this case, since the classes are homogenous and 1A is not differentiable from 1B or 2A. 

### Correcting the equality

In [114]:
def correct_equality(model, g, g_prime):
    if g == g_prime:
        return Constraint.Skip
    return quicksum(model.Assign[g, t, r] for t in model.TIMESLOTS for r in model.ROOMS) == quicksum(model.Assign[g_prime, t, r] for t in model.TIMESLOTS for r in model.ROOMS)

m.equality_rule = Constraint(m.GRADES, m.GRADES, rule = correct_equality)

(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().


In [115]:
instance = m.create_instance(instanceData)

In [116]:
opt.solve(instance, tee = True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpsnsefb_l.glpk.raw --wglp /tmp/tmpt7zsuh36.glpk.glp --cpxlp
 /tmp/tmp273yv60m.pyomo.lp
Reading problem data from '/tmp/tmp273yv60m.pyomo.lp'...
/tmp/tmp273yv60m.pyomo.lp:724: warning: lower bound of variable 'x2' redefined
/tmp/tmp273yv60m.pyomo.lp:724: warning: upper bound of variable 'x2' redefined
94 rows, 48 columns, 336 non-zeros
48 integer variables, all of which are binary
772 lines were read
Writing problem data to '/tmp/tmpt7zsuh36.glpk.glp'...
617 lines were written
GLPK Integer Optimizer 5.0
94 rows, 48 columns, 336 non-zeros
48 integer variables, all of which are binary
Preprocessing...
22 rows, 32 columns, 168 non-zeros
32 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 18
Solving LP relaxation...
GLPK Simplex Optimize

{'Problem': [{'Name': 'unknown', 'Lower bound': 12.0, 'Upper bound': 12.0, 'Number of objectives': 1, 'Number of constraints': 94, 'Number of variables': 48, 'Number of nonzeros': 336, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.007239103317260742}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [117]:
instance.display()

Model unknown

  Variables:
    Assign : Size=48, Index=GRADES*TIMESLOTS*ROOMS
        Key               : Lower : Value : Upper : Fixed : Stale : Domain
         ('1A', 8, '101') :     0 :   1.0 :     1 : False : False : Binary
         ('1A', 8, '102') :     0 :   0.0 :     1 : False : False : Binary
         ('1A', 9, '101') :     0 :   1.0 :     1 : False : False : Binary
         ('1A', 9, '102') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 10, '101') :     0 :   1.0 :     1 : False : False : Binary
        ('1A', 10, '102') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 11, '101') :     0 :   1.0 :     1 : False : False : Binary
        ('1A', 11, '102') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 12, '101') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 12, '102') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 13, '101') :     0 :   0.0 :     1 : False : False : Binary
        ('1A', 13, '1

In [118]:
from collections import defaultdict
d = defaultdict(lambda: defaultdict(str))
for g in instance.GRADES:
    for t in instance.TIMESLOTS:
        for r in instance.ROOMS:
            if value(instance.Assign[g, t, r]):
                d[r][t] = g

In [119]:
pd.DataFrame(d).fillna('N/A').sort_index()

,101,102
8,1A,1B
9,1A,2A
10,1A,1B
11,1A,2A
12,N/A,1B
13,N/A,2A
14,N/A,1B
15,N/A,2A


### Version 5

In [124]:
input_rooms = pd.read_csv("SRO_input_rooms.csv")
input_grades = pd.read_csv("SRO_input_grades.csv")

In [125]:
input_rooms

,Room_ID,Capacity
0,A121,15
1,A120,13
2,A123,15
3,A137,12
4,A138,15
5,B142,12
6,B144,12
7,B146,12
8,B148,12
9,C153,12


In [137]:
input_grades.sort_values('Population')

,Grade_ID,Population
2,PreKC,10
0,PreKA,12
1,PreKB,14
5,FirstA,15
3,KindergartenA,16
6,FirstB,16
8,ThirdA,16
9,ThirdB,17
4,KindergartenB,18
7,SecondA,32


In [128]:
instanceData = {None:{
    'GRADES': {None: input_grades['Grade_ID'].unique()},
    'TIMESLOTS': {None: range(8, 16)},
    'ROOMS': {None: input_rooms['Room_ID'].unique()},
    'pop': input_grades.set_index('Grade_ID').to_dict()['Population'],
    'cap': input_rooms.set_index('Room_ID').to_dict()['Capacity']
}}

In [129]:
instance = m.create_instance(instanceData)

In [130]:
opt.solve(instance, tee = True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmp8kpr5vjd.glpk.raw --wglp /tmp/tmp8csc7vay.glpk.glp --cpxlp
 /tmp/tmpqwxlfl6w.pyomo.lp
Reading problem data from '/tmp/tmpqwxlfl6w.pyomo.lp'...
/tmp/tmpqwxlfl6w.pyomo.lp:52966: warning: lower bound of variable 'x2' redefined
/tmp/tmpqwxlfl6w.pyomo.lp:52966: warning: upper bound of variable 'x2' redefined
2100 rows, 1728 columns, 43200 non-zeros
1728 integer variables, all of which are binary
54694 lines were read
Writing problem data to '/tmp/tmp8csc7vay.glpk.glp'...
50727 lines were written
GLPK Integer Optimizer 5.0
2100 rows, 1728 columns, 43200 non-zeros
1728 integer variables, all of which are binary
Preprocessing...
372 rows, 632 columns, 15168 non-zeros
632 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 251
Solving LP relax

{'Problem': [{'Name': 'unknown', 'Lower bound': 48.0, 'Upper bound': 48.0, 'Number of objectives': 1, 'Number of constraints': 2100, 'Number of variables': 1728, 'Number of nonzeros': 43200, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.051724910736083984}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [132]:
from collections import defaultdict
d = defaultdict(lambda: defaultdict(str))
for g in instance.GRADES:
    for t in instance.TIMESLOTS:
        for r in instance.ROOMS:
            if value(instance.Assign[g, t, r]):
                d[r][t] = g

In [133]:
pd.DataFrame(d).fillna('N/A').sort_index()

,A121,A137,A123,A138,A120,C155,T2,T3,T1
8,PreKA,N/A,PreKB,FirstA,PreKC,KindergartenA,ThirdB,ThirdA,SecondA
9,FirstA,PreKA,PreKB,N/A,PreKC,ThirdB,FirstB,FifthA,FourthA
10,FirstA,N/A,PreKA,PreKB,PreKC,FirstB,SecondA,KindergartenB,FourthA
11,PreKB,PreKA,FirstA,N/A,PreKC,KindergartenB,KindergartenA,FifthA,ThirdA
12,N/A,N/A,N/A,N/A,N/A,ThirdA,ThirdB,SecondA,FirstB
13,N/A,N/A,N/A,N/A,N/A,KindergartenA,KindergartenB,FifthA,FourthA
14,N/A,N/A,N/A,N/A,N/A,KindergartenA,KindergartenB,SecondA,FourthA
15,N/A,N/A,N/A,N/A,N/A,ThirdA,ThirdB,FifthA,FirstB


In [138]:
face_to_face_time_third_B = 0
for t in instance.TIMESLOTS:
    for r in instance.ROOMS:
        face_to_face_time_third_B += value(instance.Assign['ThirdB', t, r])

In [139]:
max_hours = face_to_face_time_third_B * input_grades['Population'].sum()

In [140]:
max_hours

np.float64(952.0)